### Introduction

#### Code to summarize event counts and parse JSON event data for the 2019 Data Science Bowl Sponsored by Kaggle: https://www.kaggle.com/c/data-science-bowl-2019

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import json

#### Using Python functions to make the code easier to read and understand

In [2]:
#helpers for summarizing event data
def GetAllSessionsFromFile(filename):
    df_sessions = pd.read_csv(filename);
    return df_sessions;

def PrintMismatchesInEventCounts (session_id, df_file, df_transformed):
    total_events_from_file = len(df_file[df_file.game_session == session_id])
    total_events_after_transform = df_transformed[df_transformed.game_session == session_id].total_events.values[0]
    if (total_events_from_file != total_events_after_transform ):
        print("mismatch for " + session_id)
        print("Found in file = " + str(total_events_from_file) + " Found after transform = " + str(total_events_after_transform))

In [3]:
#helpers to parse JSON data and to calculate the ground truth 
assessment_default = 4100;
assessment_bird = 4110; 

def IsCorrectAssessment(jsonData):
    for key, value in jsonData.items():
        if 'correct' in key:
            return value

def CalcAccuracyGroup(row):
    if row['num_correct'] == 0:
        return 0
    elif row['attempts'] == 1:
        return 3
    elif row ['attempts'] == 2:
        return 2
    elif row ['attempts'] >= 3:
        return 1

def VerifyCalculationsMatchGroundTruth(listCalculatedAccuracyGroup):
    #Compare the accuracy_group calculated with the ones provided in the label file
    labels = pd.read_csv('data/train_labels.csv')
    labels_accuracy_group = labels['accuracy_group'].values.tolist()
    return (labels_accuracy_group == listCalculatedAccuracyGroup)


#### Determining how many different kinds of events are in the test file

In [4]:
df_file = GetAllSessionsFromFile('data/train.csv')
print("Unique event codes from the file:")
print(df_file.event_code.unique().tolist())

Unique event codes from the file:
[2000, 3010, 3110, 4070, 4090, 4030, 4035, 4021, 4020, 4010, 2080, 2083, 2040, 2020, 2030, 3021, 3121, 2050, 3020, 3120, 2060, 2070, 4031, 4025, 5000, 5010, 2081, 2025, 4022, 2035, 4040, 4100, 2010, 4110, 4045, 4095, 4220, 2075, 4230, 4235, 4080, 4050]


#### For each user session, sum the events of each category and sum the total number of events.  Since Pandas DataFrame column names must be valid Python variable names, the column names cannot start with a number.  Therefore, prefix each column name with some text.

In [5]:
groupby1 = ['game_session', 'installation_id', 'world', 'event_code']
#planning to use df_file later in notebook, so creating a new dataframe 
summary = df_file.groupby(groupby1)['event_count'].count().unstack().add_prefix('event_count_').fillna(0)
summary = summary.reset_index()
summary.head()


event_code,game_session,installation_id,world,event_count_2000,event_count_2010,event_count_2020,event_count_2025,event_count_2030,event_count_2035,event_count_2040,...,event_count_4080,event_count_4090,event_count_4095,event_count_4100,event_count_4110,event_count_4220,event_count_4230,event_count_4235,event_count_5000,event_count_5010
0,00000c4b5e2c400f,3ad8a814,TREETOPCITY,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000050630c4b081b,b57e4a95,MAGMAPEAK,1.0,0.0,4.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00005be8058d8e35,fb91172c,CRYSTALCAVES,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0000b2837f00d9a4,100b0f77,NONE,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0000d11e30a5391c,cfabf670,CRYSTALCAVES,1.0,0.0,9.0,0.0,8.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Adding a column for the total number of events, so check to see which way is the most efficient.  Make it a fair test, so drop the newly added total_events column after each trial.

In [6]:
%%time
summary['total_events'] = summary.sum(axis=1)

CPU times: user 3.71 s, sys: 193 ms, total: 3.9 s
Wall time: 2.63 s


In [7]:
summary.drop('total_events', 1)
print('just dropped total_events')

just dropped total_events


In [8]:
%%time
summary.loc[:,'total_events'] = summary.sum(axis=1)

CPU times: user 3.93 s, sys: 180 ms, total: 4.11 s
Wall time: 2.71 s


In [9]:
summary.drop('total_events', 1)
print('just dropped total_events')

just dropped total_events


In [10]:
# a new dataframe is not required, so .assign is not necessary
#summary.assign(total_events = summary.sum(axis=1))


#### Choosing loc because it is the fastest and creating a new dataframe isn't necessary

In [11]:
summary.loc[:, 'total_events'] = summary.loc[:, 'event_count_2000':'event_count_5010'].sum(axis=1)
summary.head()


event_code,game_session,installation_id,world,event_count_2000,event_count_2010,event_count_2020,event_count_2025,event_count_2030,event_count_2035,event_count_2040,...,event_count_4090,event_count_4095,event_count_4100,event_count_4110,event_count_4220,event_count_4230,event_count_4235,event_count_5000,event_count_5010,total_events
0,00000c4b5e2c400f,3ad8a814,TREETOPCITY,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,000050630c4b081b,b57e4a95,MAGMAPEAK,1.0,0.0,4.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.0
2,00005be8058d8e35,fb91172c,CRYSTALCAVES,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0000b2837f00d9a4,100b0f77,NONE,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0000d11e30a5391c,cfabf670,CRYSTALCAVES,1.0,0.0,9.0,0.0,8.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130.0


#### Assumption checks to make sure the data is looking how it should.  While the number of events in the file seem to be matching the number of events after summarizing the data, some of the Worlds are NONE.  Still deciding what this means and how to handle them.

In [12]:
PrintMismatchesInEventCounts('8fdd5d389d0e272e', df_file, summary)
PrintMismatchesInEventCounts('67f9274eb736ab3f', df_file, summary)
PrintMismatchesInEventCounts('0000d11e30a5391c', df_file, summary)

print("Unique Worlds after processing:")
print(summary.world.unique().tolist())

Unique Worlds after processing:
['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES', 'NONE']


#### The goal of the Data Science Bowl is to predict how well a user will do on an Assessment.  The competition provided a test file, a train file, and a label file that has the actual user score (accuracy_group) for the users in test file.  To determine a user's actual score, the event_data (which is in JSON format) needs to be parsed and then some calculations made based on how often the user correctly answered the Assessment.   Therefore, it is necessary to be able to determine a user's score (accuracy_group) for the competition and a good starting point is to calculate the score (accuracy_group) for the users in the test file and compare that to the label file.    This can be thought of as "finding the ground truth."   


In [13]:
#get all the assessments in the file.  use .copy(deep=True) to prevent warnings
df_assessments = df_file[  ( (df_file.event_code == assessment_bird) & (df_file.title.str.contains("Bird Measurer (Assessment)", regex=False)) )
                            | ( (df_file.event_code == assessment_default) & (df_file.title.str.contains("(Assessment)", regex=False)) & (~df_file.title.str.contains("Bird Measurer (Assessment)", regex=False) ) )].copy(deep=True) 
    


In [14]:
#turn the JSON string into a Python object and put it in a new column
df_assessments.loc[:, 'event_json'] = df_assessments['event_data'].apply(json.loads)

#based on the JSON data, determine if the Assessment was answered correctly 
df_assessments.loc[:, 'is_correct'] = df_assessments.apply(lambda row: IsCorrectAssessment(row['event_json']), axis=1)

df_assessments.head(1)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,event_json,is_correct
2228,25fa8af4,901acc108f55a5a1,2019-08-06T05:22:32.357Z,"{""correct"":true,""stumps"":[1,2,4],""event_count"":44,""game_time"":31011,""event_code"":4100}",0006a69f,44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,"{'correct': True, 'stumps': [1, 2, 4], 'event_count': 44, 'game_time': 31011, 'event_code': 4100}",True


In [15]:
#make the calculations and set the accuracy_group (which is the prediction needed for the competition)

relevant_cols = ['event_id', 'game_session', 'installation_id', 
                'title', 'is_correct', 'world', 'game_time', 'timestamp']
groupby_list = ['game_session', 'installation_id', 'title', 'world']

df_assessments = df_assessments[relevant_cols]
result = df_assessments.groupby(groupby_list, as_index=False) \
    .apply(lambda row: pd.Series({
      'attempts'      : row['is_correct'].count(),
      'num_correct'   : (row['is_correct'] == True).sum(),
      'num_incorrect' : (row['is_correct'] == False).sum()
  })
).reset_index()
 
result['accuracy'] = result['num_correct']/result['attempts']
result['accuracy_group'] = result.apply(lambda row: CalcAccuracyGroup(row), axis=1)
result = result.sort_values(['installation_id', 'game_session'])

result.head()

,game_session,installation_id,title,world,attempts,num_correct,num_incorrect,accuracy,accuracy_group
7401,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),TREETOPCITY,1,1,0,1.0,3
8270,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),TREETOPCITY,11,0,11,0.0,0
9874,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),TREETOPCITY,1,1,0,1.0,3
10235,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),TREETOPCITY,2,1,1,0.5,2
11714,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),TREETOPCITY,1,1,0,1.0,3


In [16]:
VerifyCalculationsMatchGroundTruth(result['accuracy_group'].values.tolist())

True

#### Summary:   The accuracy_group calculated in the notebook matches the file provided.  This means the logic to calculate the accuracy_group is correct and can be used for competition submissions. 
